In [1]:
import pandas as pd

In [1]:
def to_sql(table,df):
    import pandas as pd
    from sqlalchemy import create_engine


    # MySQL connection parameters
    mysql_user = "root"
    mysql_password = "Sharath45"
    mysql_host = "127.0.0.1"
    mysql_port = 3306  # Default MySQL port
    mysql_database = "jorie_ohdsi"

    mysql_table = table

    # Create a connection engine
    engine = create_engine(f"mysql+pymysql://{mysql_user}:{mysql_password}@{mysql_host}:{mysql_port}/{mysql_database}")

    # Updating the MySQL table
    try:
        # Append data to the existing table while matching column names
        df.to_sql(name=mysql_table, con=engine, if_exists='replace', index=False)
        print("Data updated successfully into the MySQL table.")
    except Exception as e:
        print(f"An error occurred: {e}")


In [ ]:

Patients=pd.read_excel(r"PatientDemo.xlsx")
Encounters=pd.read_excel(r"PatientEncounter.xlsx")
Lab_results=pd.read_excel("PatientLabResult.xlsx")
conditions=pd.read_excel("PatientDiagnosis.xlsx")
procedures=pd.read_excel("PatientProcedure.xlsx")


Creating Location Table

In [3]:
# Rename columns as specified
Location=Patients[['ZIP', 'Address', 'FIPS','MRN']]
Location = Location.rename(columns={
    'MRN': 'location_id',
    'ZIP': 'zip',
    'Address': 'address_1',
    'FIPS': 'county'
})


Location['address_2'] = None  
Location['city'] = None #Need to check weather we can extract from address
Location['state'] = None   #Need to check weather we can extract from address
Location['location_source_value'] = None 

# Keep all other columns in the DataFrame as well
# (this step ensures that all existing columns are retained)
Location = Location[['location_id', 'address_1', 'address_2', 'city', 'state', 'zip', 'county', 'location_source_value']]

In [9]:
Location=Location.drop_duplicates()

In [10]:
Location

,location_id,address_1,address_2,city,state,zip,county,location_source_value
0,E223702,3083 N GEORGIA ST,None,None,None,46760,NaN,None
2,E371944,5858 N 600 E,None,None,None,46731,NaN,None
4,E435319,PO BOX 145,None,None,None,46974-0145,NaN,None
6,E475126,1543 TALLGRASS TRL,None,None,None,46804,NaN,None
8,E494826,6321 BENNINGTON DR,None,None,None,46815,NaN,None
...,...,...,...,...,...,...,...,...
2443,E994516,1861 GLENLAKE CT,None,None,None,60451,NaN,None
2444,E994965,142 NORTH DRIVE,None,None,None,46740,NaN,None
2445,E99509,525 N BROADWAY ST,None,None,None,46721,NaN,None
2446,E99554,3425 NEW HAVEN AVE,None,None,None,46803,NaN,None


In [ ]:

mysql_table = "location"

# Create a connection engine
engine = create_engine(f"mysql+pymysql://{mysql_user}:{mysql_password}@{mysql_host}:{mysql_port}/{mysql_database}")

# Updating the MySQL table
try:
    # Append data to the existing table while matching column names
    Location.to_sql(name=mysql_table, con=engine, if_exists='replace', index=False)
    print("Data updated successfully into the MySQL table.")
except Exception as e:
    print(f"An error occurred: {e}")


Data updated successfully into the MySQL table.


Creating Death Table

In [13]:
Death=Patients[["MRN","DOD"]]
Death = Death.rename(columns={
    'MRN': 'person_id',
    'DOD': 'death_date',
})
Death['death_datetime'] = Death["death_date"] 
Death['death_type_concept_id'] = None  
Death['cause_concept_id'] = None  
Death['cause_source_value'] = None
Death['cause_source_concept_id'] = None


Creating Person Table

In [15]:
Person=Patients[["DOB",'Ethnicity', 'MRN','Race',"Gender"]]
# Person["DOB"]=pd.to_datetime(Person["DOB"])
Person["year_of_birth"]=Person["DOB"]
Person["day_of_birth"]=Person["DOB"]
Person["month_of_birth"]=Person["DOB"]
Person["birth_datetime"]=Person["DOB"]
Person["ethnicity_source_value"]=Person["Ethnicity"]
Person["race_source_value"]=Person["Race"]
Person["person_id"]=Person["MRN"]
Person["location_id"]=Person["MRN"]
Person["gender_source_value"]=Person["Gender"]
Person["gender_concept_id"]=None
Person["race_concept_id"] = None
Person["ethnicity_concept_id"] = None
Person["provider_id"] = None
Person["care_site_id"] = None
Person["person_source_value"] = None
Person["gender_source_concept_id"] = None
Person["race_source_concept_id"] = None
Person["ethnicity_source_concept_id"] = None

Person=Person.drop(columns=["DOB",'Ethnicity', 'MRN','Race',"Gender"])


C:\Users\Pc\AppData\Local\Temp\ipykernel_15968\709119142.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Person["year_of_birth"]=Person["DOB"]
C:\Users\Pc\AppData\Local\Temp\ipykernel_15968\709119142.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Person["day_of_birth"]=Person["DOB"]
C:\Users\Pc\AppData\Local\Temp\ipykernel_15968\709119142.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

Data updated successfully into the MySQL table.


Creating Visit occurance

In [20]:
Encounters.columns

Index(['Encounter', 'Encounter ID', 'Encounter Type', 'Encounter DOS',
       'Facility ID', 'Attending Physician', 'Facility me', 'DRG', 'MRN',
       'Patient me', 'Encounter Reason'],
      dtype='object')

In [21]:
Visit_occurance=Encounters[[ 'Encounter ID', 'Encounter Type', 'Encounter DOS',"MRN",
       'Attending Physician', 'Facility me', 'Encounter Reason']]
column_mapping = {
    'Encounter ID': 'visit_occurrence_id',
    'MRN': 'person_id',
    'Encounter DOS': 'visit_start_date',
    # 'Admit Time': 'visit_start_datetime',
    # 'Discharge Time': 'visit_end_datetime',
    'Encounter Type': 'visit_type_concept_id',
    'Attending Physician': 'provider_id',
    'Facility me': 'care_site_id',
"Encounter Reason":"visit_source_value"

    # 'Discharge Disposition': 'discharge_to_concept_id'
}
Visit_occurance.rename(columns=column_mapping, inplace=True)
Visit_occurance["visit_start_datetime"] = Visit_occurance["visit_start_date"]
Visit_occurance["discharge_to_concept_id"] = None
Visit_occurance["visit_concept_id"] = None
Visit_occurance["visit_end_date"] = None
Visit_occurance["visit_end_datetime"] = None
Visit_occurance["admitted_from_concept_id"] = None
Visit_occurance["preceding_visit_occurrence_id"] = None
Visit_occurance["visit_source_concept_id"] = None
Visit_occurance["admitted_from_source_value"] = None
Visit_occurance["discharge_to_source_value"] = None
Visit_occurance["discharged_to_concept_id"] = None


C:\Users\Pc\AppData\Local\Temp\ipykernel_15968\2224114910.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Visit_occurance.rename(columns=column_mapping, inplace=True)
C:\Users\Pc\AppData\Local\Temp\ipykernel_15968\2224114910.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Visit_occurance["visit_start_datetime"] = Visit_occurance["visit_start_date"]
C:\Users\Pc\AppData\Local\Temp\ipykernel_15968\2224114910.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [23]:
to_sql("visit_occurrence",Visit_occurance)

Data updated successfully into the MySQL table.


Creating Observation Table

In [24]:
Observations=Patients[["MRN",'Alcohol', 'Smoking',]]

Observations = Observations.melt(
    id_vars=["MRN"], 
    value_vars=["Alcohol", "Smoking"], 
    var_name="Categorey", 
    value_name="Value"
)

# Renaming columns as per the mapping
column_mapping_observations = {
    "MRN": "person_id",
    "Categorey": "observation_type_concept_id",
    "Value": "value_as_string"
}

Observations.rename(columns=column_mapping_observations, inplace=True)

In [25]:
Observations=Observations.drop_duplicates()

In [26]:
to_sql("observation",Observations)

Data updated successfully into the MySQL table.


Creating Observations Table

In [34]:
measurements=Lab_results[["MRN",'Order date','Specimen type-like blood, urnine','Result value','Units','Normal range',
                         'Encounter ID','LOINC Code']]
rename_mapping_measurements = {
    'Order date': 'measurement_date',
    'Specimen type-like blood, urnine': 'measurement_type_concept_id',
    'Result value': 'value_as_number',
    'Units': 'unit_concept_id',
    'Normal range': 'range_low',  # Will require splitting into two columns
    'Encounter ID': 'visit_occurrence_id',
    'LOINC Code': 'measurement_source_value',
    'MRN':"person_id"
}

# Renaming columns (for Normal range, split into two new columns)
measurements.rename(columns=rename_mapping_measurements, inplace=True)

measurements['range_high']=measurements["range_low"]

C:\Users\Pc\AppData\Local\Temp\ipykernel_15968\3452461889.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  measurements.rename(columns=rename_mapping_measurements, inplace=True)
C:\Users\Pc\AppData\Local\Temp\ipykernel_15968\3452461889.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  measurements['range_high']=measurements["range_low"]


In [35]:
to_sql("measurement",measurements)

Data updated successfully into the MySQL table.


Creating Conditions Occurance

In [29]:
conditions.columns

Index(['Diagnosis', 'Patient me', 'MRN', 'DX Timestamp', 'Facility ID',
       'Primary Diagnosis', 'HCC Conditions', 'Encounter DOS',
       'Encounter Type'],
      dtype='object')

In [31]:
conditions_transfroamtion=conditions[["MRN","Primary Diagnosis","HCC Conditions"	,"Encounter DOS",'Encounter Type']]

In [32]:

conditions_transfroamtion = conditions_transfroamtion.melt(
    id_vars=["MRN","Encounter DOS",'Encounter Type'], 
    value_vars=["Primary Diagnosis","HCC Conditions"], 
    var_name="Categorey", 
    value_name="Value"
)


In [33]:
conditions_transfroamtion=conditions_transfroamtion.drop_duplicates()

In [34]:
conditions_transfroamtion['Value'] = conditions_transfroamtion['Value'].str.split(', ')
conditions_transfroamtion = conditions_transfroamtion.explode('Value')

In [35]:
conditions_transfroamtion.columns

Index(['MRN', 'Encounter DOS', 'Encounter Type', 'Categorey', 'Value'], dtype='object')

In [ ]:

# person_id-MRN
# condition_start_date-Encounter DOS
# condition_start_datetime-Encounter DOS
# condition_type_concept_id-Value
# condition_source_value--Categorey


In [36]:
conditons_occurence=conditions_transfroamtion[['MRN', 'Encounter DOS','Encounter Type' ,'Categorey', 'Value']]
conditons_occurence = conditons_occurence.rename(columns={
    'MRN': 'person_id',
    'Encounter DOS': 'condition_start_datetime',
    'Categorey': 'condition_source_value',
    'Value': 'condition_type_concept_id'
})


In [37]:
to_sql("condition_occurrence",conditons_occurence)

Data updated successfully into the MySQL table.


In [9]:
conditons_occurence


,person_id,condition_start_datetime,condition_source_value,condition_type_concept_id
0,E461345,2025-04-01,Primary Diagnosis,D64.9
0,E461345,2025-04-01,Primary Diagnosis,Z12.11
1,E461345,2025-04-01,Primary Diagnosis,NaN
2,E1266972,2025-03-24,Primary Diagnosis,NaN
3,E1266972,2025-03-18,Primary Diagnosis,NaN
...,...,...,...,...
34026,E190284,2004-11-07,HCC Conditions,E11.9
34027,E173208,2004-06-18,HCC Conditions,NaN
34028,E41554,2004-05-03,HCC Conditions,NaN
34029,E534155,2004-04-16,HCC Conditions,E11.65


In [11]:
diab_codes=pd.read_excel(r"C:\Users\Pc\OneDrive - Aidwise Private Ltd\Jorie\ICD10_diabetes_codes.xlsx")

In [12]:
diab_codes.columns

Index(['ICD_10', 'DESCRIPTIONS', 'Code Status'], dtype='object')

In [13]:
Diabetic_patietns=conditons_occurence[conditons_occurence["condition_type_concept_id"].isin(diab_codes["ICD_10"].unique())]

In [15]:
Diabetic_patietns["person_id"].nunique()

472

In [17]:
Diabetic_patietns["condition_type_concept_id"].unique()

array(['E11.621', 'E11.42', 'E11.40', 'E13.21', 'E11.9', 'E11.69',
       'E11.3411', 'E11.65', 'E11.22', 'E10.65', 'E10.10', 'E11.43',
       'E08.59', 'E11.51', 'E11.8', 'E11.628', 'E11.610', 'E11.59',
       'E10.9', 'E13.40', 'E11.3599', 'E11.319', 'E11.3292', 'E11.49',
       'E11.00', 'E11.622', 'E10.22', 'E08.621', 'E11.618', 'E11.3293',
       'E11.21', 'E11.29', 'E11.311', 'E11.3419', 'E11.3512', 'E11.3593',
       'E11.10', 'E11.3553', 'E11.3513', 'E11.3299', 'E11.3291',
       'E11.649', 'E13.9', 'E13.69', 'E10.621', 'E10.49', 'E10.8',
       'E10.649', 'E13.621', 'E10.69', 'E11.3393', 'E13.51', 'E11.01',
       'E11.52', 'E10.3553', 'E11.39'], dtype=object)

In [19]:
diab_codes[diab_codes["ICD_10"].isin(Diabetic_patietns["condition_type_concept_id"].unique())].to_csv("Diabetic_patients.csv")

Creating Procedures

In [ ]:
import pandas as pd


In [39]:
procedures.columns

Index(['MRN', 'Procedure', 'Encounter Type', 'Encounter DOS', 'Patient me',
       'Facility ID', 'Performing Provider', 'Mapped Proc Code',
       'Ordering Provider id', 'local me- SNOMED descp'],
      dtype='object')

In [40]:
procedures=procedures[['MRN','Encounter Type','Encounter DOS', 'Mapped Proc Code']]

In [41]:
procedures['Mapped Proc Code'] = procedures['Mapped Proc Code'].str.split(', ')
procedures = procedures.explode('Mapped Proc Code')

In [42]:
procedures.drop_duplicates()

,MRN,Encounter Type,Encounter DOS,Mapped Proc Code
0,E655096,Surgery,11/04/2024,NaN
1,E558187,Surgery,09/20/2019,NaN
2,E428253,Surgery,02/28/2023,NaN
3,E221550,Surgery,11/01/2023,NaN
4,E671480,Surgery,06/23/2015,NaN
...,...,...,...,...
54437,E692445,Lab,06/09/2023,NaN
54438,E692445,Office Visit,05/30/2023,NaN
54439,E692445,Travel,05/30/2023,NaN
54440,E692445,Office Visit,04/11/2023,NaN


In [43]:
# procedure_occurrence_id
# person_id-MRN
# procedure_concept_id-Encounter Type
# procedure_date-Encounter DOS
# procedure_datetime
# procedure_end_date
# procedure_end_datetime
# procedure_type_concept_id
# modifier_concept_id
# quantity
# provider_id
# visit_occurrence_id
# visit_detail_id
# procedure_source_value
# procedure_source_concept_id
# modifier_source_value


procedures = procedures.rename(columns={
    'MRN': 'person_id',
    'Encounter DOS': 'procedure_date',
    'procedure_concept_id': 'Mapped Proc Code',
    'Encounter Type': 'Encounter Type'
})

In [44]:
procedures

,person_id,Encounter Type,procedure_date,Mapped Proc Code
0,E655096,Surgery,11/04/2024,NaN
1,E558187,Surgery,09/20/2019,NaN
2,E428253,Surgery,02/28/2023,NaN
3,E221550,Surgery,11/01/2023,NaN
4,E671480,Surgery,06/23/2015,NaN
...,...,...,...,...
54437,E692445,Lab,06/09/2023,NaN
54438,E692445,Office Visit,05/30/2023,NaN
54439,E692445,Travel,05/30/2023,NaN
54440,E692445,Office Visit,04/11/2023,NaN


In [45]:
to_sql("procedure_occurrence",procedures)

Data updated successfully into the MySQL table.
